In [1]:
# Standard Libraries
import os
from pathlib import Path

# Third party libraries
import gym
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.ppo.policies import MlpPolicy #  MlpPolicy because the observation of the CartPole task is a feature vector, not images.
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv

In [2]:
# def evaluate(model, num_episodes=100):
#     """
#     Evaluate a RL agent
#     :param model: (BaseRLModel object) the RL Agent
#     :param num_episodes: (int) number of episodes to evaluate it
#     :return: (float) Mean reward for the last num_episodes
#     """
#     # This function will only work for a single Environment
#     env = model.get_env()
#     all_episode_rewards = []
#     for i in range(num_episodes):
#         episode_rewards = []
#         done = False
#         obs = env.reset()
#         while not done:
#             # _states are only useful when using LSTM policies
#             action, _states = model.predict(obs)
#             # here, action, rewards and dones are arrays
#             # because we are using vectorized env
#             obs, reward, done, info = env.step(action)
#             episode_rewards.append(reward)

#         all_episode_rewards.append(sum(episode_rewards))

#     mean_episode_reward = np.mean(all_episode_rewards)
#     print("Mean reward:", mean_episode_reward, "Num episodes:", num_episodes)

#     return mean_episode_reward

# Initialization

In [3]:
env_name = 'CartPole-v1'
# Create folder to save models
directory_path = 'models'
Path(directory_path).mkdir(parents=True, exist_ok=True)

## Create model

In [4]:
env = gym.make(env_name)
model = PPO(MlpPolicy, env, verbose=0)

# Training

In [5]:
# Train the agent for n steps
n_steps = 100_000
model = model.learn(total_timesteps=n_steps)

## Save model

In [6]:
model_file_name = Path(directory_path, env_name+'_'+str(n_steps))
model.save(model_file_name)

In [7]:
# sample an observation from the environment
obs = model.env.observation_space.sample()

# Check prediction before saving
print("pre saved", model.predict(obs, deterministic=True))

del model # delete trained model to demonstrate loading

pre saved (0, None)


In [8]:
loaded_model = PPO.load(model_file_name)
# Check that the prediction is the same after loading (for the same observation)
# print("loaded", loaded_model.predict(obs, deterministic=True))

# show the save hyperparameters
print("loaded:", "gamma =", loaded_model.gamma, "n_steps =", loaded_model.n_steps)

loaded: gamma = 0.99 n_steps = 2048


In [9]:
# as the environment is not serializable, we need to set a new instance of the environment
loaded_model.set_env(DummyVecEnv([lambda: gym.make(env_name)]))
# and continue training
model = loaded_model.learn(n_steps)

# Evaluation

In [10]:
# mean_episode_reward = evaluate(model, num_episodes=100)
# print(mean_episode_reward)

In [11]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

/Users/mattiacinelli/repositories/AlgoRL/.valgorl/lib/python3.8/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean_reward:500.00 +/- 0.00


In [17]:
# Set up fake display; otherwise rendering will fail
import os
# os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [18]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay

def show_videos(video_path='', prefix=''):
    """
    Taken from https://github.com/eleurent/highway-env

    :param video_path: (str) Path to the folder containing videos
    :param prefix: (str) Filter the video, showing only the only starting with this prefix
    """
    html = []
    for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append('''<video alt="{}" autoplay loop controls style="height: 400px;">
        <source src="data:video/mp4;base64,{}" type="video/mp4" />
        </video>'''.format(mp4, video_b64.decode('ascii')))
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

In [19]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(env_id, model, video_length=500, prefix='', video_folder='videos/'):
    """
    :param env_id: (str)
    :param model: (RL model)
    :param video_length: (int)
    :param prefix: (str)
    :param video_folder: (str)
    """
    eval_env = DummyVecEnv([lambda: gym.make(env_id)])
    # Start the video at step=0 and record 500 steps
    eval_env = VecVideoRecorder(
        eval_env, video_folder=video_folder,
        record_video_trigger=lambda step: step == 0, video_length=video_length,
        name_prefix=prefix)
        
    obs = eval_env.reset()
    for _ in range(video_length):
        action, _ = model.predict(obs)
        obs, _, _, _ = eval_env.step(action)

  # Close the video recorder
    eval_env.close()

## Visualize trained agent

In [20]:
record_video('CartPole-v1', model, video_length=500, prefix='ppo2-cartpole')

2022-03-08 16:05:22.794 Python[18658:529692] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to (null)


Saving video to /Users/mattiacinelli/repositories/AlgoRL/algorl/gym_examples/classics/videos/ppo2-cartpole-step-0-to-step-500.mp4


In [21]:
show_videos('videos', prefix='ppo2')